In [1]:
import pyvista as pv
import numpy as np
import os, flopy
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation, LinearTriInterpolator
from tqdm import tqdm

    
name = 'Model'
workspace = '../model'
mf_exe_name = '../exe/mf6'

sim = flopy.mf6.MFSimulation.load(sim_name=name, exe_name=mf_exe_name, sim_ws=workspace)

mfmodel = sim.get_model(model_name='model')

fname = os.path.join(workspace, name + '.hds')
hdobj = flopy.utils.HeadFile(fname, precision='double')
head = hdobj.get_data()
headRshp = head.reshape((head.shape[0],head.shape[2]))

flopy is installed in /home/gida2/.local/lib/python3.6/site-packages/flopy
loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package disv...
    loading package npf...
    loading package ic...
    loading package rch...
    loading package evt...
    loading package drn...
    loading package oc...
  loading ims package model...


In [2]:
# define vertex interpolation functions
def triInterpolation(zValue, xPoints, yPoints):
    triObj = Triangulation(xPoints, yPoints)
    fz = LinearTriInterpolator(triObj, zValue)
    return fz

def interpVerticesZ(zName, zValues, xyCentroid, xyVertices):
    interpVerticesList=[]
    layTri = triInterpolation(zValues, xyCentroid[:,0], xyCentroid[:,1])
    
    for index, vertice in tqdm(enumerate(xyVertices), desc="Working vertices for %s"%zName):
        
        newZ = layTri(vertice[0],vertice[1])
        if newZ.mask:
            for index2, row2 in enumerate(cell2dList):
                if index in row2[1:]:
                    newZ = zValues[index2]
        interpVerticesList.append(newZ)
    interpVerticesArray = np.array(interpVerticesList)
    return interpVerticesArray

def interpVerticesHeads(zName, zValues, xyCentroid, xyVertices):
    interpVerticesList=[]
    #filtering over real heads
    realHeadIndexArray = (zValues>-1.00000000e+30).nonzero()[0]
    #reassigning arrays
    zValuesFiltered = zValues[realHeadIndexArray]
    xyCentroidFiltered = xyCentroid[realHeadIndexArray]
    verticesIndexFiltered = []
    for headCell in realHeadIndexArray:
        verticesIndexFiltered+=cell2dList[headCell][1:]
    verticesIndexFiltered = list(set(verticesIndexFiltered))
    #xyVerticesFiltered = xyVertices[cell2dListFiltered]
    #print(xyVerticesFiltered)
    #filtered triangle interpolation
    layTri = triInterpolation(zValuesFiltered, xyCentroidFiltered[:,0], xyCentroidFiltered[:,1])
    
    interpVerticesArray = np.ones([nvert])*-1.00000000e+30
    for vertexIndex in tqdm(verticesIndexFiltered, desc="Working vertices for %s"%zName):
        vertice = xyVertices[vertexIndex]
        newZ = layTri(vertice[0],vertice[1])
        if newZ.mask:
            for index2, row2 in enumerate(cell2dList):
                if vertexIndex in row2[1:]:
                    newZ = zValues[index2]
        interpVerticesArray[vertexIndex] = newZ
    return interpVerticesArray

In [4]:
# define model variables from flopy instanes
nlay = mfmodel.disv.nlay.array
nvert = mfmodel.disv.nvert.array
ncpl = mfmodel.disv.ncpl.array
modelCellZ = np.vstack((mfmodel.disv.top.array,mfmodel.disv.botm.array)) #cell Z in all layers
modelHeadZ = np.vstack([headRshp,headRshp[-1]])
# XY array for all cellvertices and centroids
verticesXYArray = np.dstack((mfmodel.disv.vertices.array.xv,mfmodel.disv.vertices.array.yv))[0] 
centroidXYArray = np.dstack((mfmodel.disv.cell2d.array.xc,mfmodel.disv.cell2d.array.yc))[0]
# XYZ for cell centroid
#centroidXYZArray = np.zeros((nlay,ncpl))
centroidXYZDict = {}
for lay in range(nlay):
    centroidXYZDict[str(lay)] = np.dstack((mfmodel.disv.cell2d.array.xc,mfmodel.disv.cell2d.array.yc,(modelCellZ[lay] + modelCellZ[lay+1])/2))[0]
#centroidXYZArray = np.dstack((mfmodel.disv.cell2d.array.xc,mfmodel.disv.cell2d.array.yc,centroidZArray))[0]

cell2dList = []
for index, cell in enumerate(mfmodel.disv.cell2d.array):
    vertexIndexList = [x for x in list(cell)[3:] if x is not None]
    cell2dList.append(vertexIndexList)        
    
# calculate cell vertex Z
print('Calculating vertices Z values')
modelVertZ = np.zeros((nlay+1,nvert))
for i in range(nlay+1):
    modelVertZ[i] = interpVerticesZ('Lay '+str(i),modelCellZ[i], centroidXYArray, verticesXYArray)
    
# calculate cell vertex head
print('Calculating vertices head values')
modelVertHead = np.zeros((nlay+1,nvert))
for i in range(nlay+1):
    modelVertHead[i] = interpVerticesHeads('Lay '+str(i),modelHeadZ[i], centroidXYArray, verticesXYArray)
    
cell2dArray = np.hstack(cell2dList)

Calculating vertices Z values


Working vertices for Lay 0: 99154it [00:08, 11835.79it/s]
Working vertices for Lay 1: 99154it [00:08, 11530.29it/s]
Working vertices for Lay 2: 99154it [00:08, 11562.61it/s]
Working vertices for Lay 3: 99154it [00:08, 11560.86it/s]
Working vertices for Lay 4: 99154it [00:08, 11697.74it/s]
Working vertices for Lay 5: 99154it [00:08, 11514.47it/s]
Working vertices for Lay 6: 99154it [00:08, 11625.87it/s]
Working vertices for Lay 0:  14%|█▍        | 1745/12171 [00:00<00:00, 17442.66it/s]

Calculating vertices head values


Working vertices for Lay 6: 100%|██████████| 99154/99154 [00:08<00:00, 12239.60it/s]


### for model geometry

In [6]:
baseGridXYZ = np.hstack((verticesXYArray,np.zeros([nvert,1])))
baseGrid = pv.PolyData(baseGridXYZ,cell2dArray)

layerTubes = {}
for lay in tqdm(range(nlay),desc=("Working geometries for model grid")):
    workingGrid = baseGrid.copy()
    workingVolume = workingGrid.extrude([0,0,1])
    for vert in range(nvert):
        workingVolume.points[vert][2] = modelVertZ[lay,vert]
        workingVolume.points[vert+nvert][2] = modelVertZ[lay+1,vert]
    layerTubes[str(lay)] = workingVolume
layerBlocks = pv.MultiBlock(layerTubes)
totalModelGrid = layerBlocks.combine()
totalModelGrid.save('../vtk/totalModelGrid.vtk')

print('Working on the outer model shell')
workingGrid = baseGrid.copy()
workingVolume = workingGrid.extrude([0,0,1])
for vert in range(nvert):
    workingVolume.points[vert][2] = modelVertZ[0,vert]
    workingVolume.points[vert+nvert][2] = modelVertZ[nlay,vert]
workingVolume.save('../vtk/modelOuterShell.vtk')

Working geometries for model grid: 100%|██████████| 6/6 [00:24<00:00,  4.07s/it]


Working on the outer model shell


In [17]:
layerTubes = {}
for lay in tqdm(range(nlay),desc=("Working head cell for model grid")):
    workingGrid = baseGrid.copy()
    #filtering cell heads
    layCellHeads = headRshp[lay]
    #layCellHeads[layCellHeads == -1.00000000e+30] = np.nan

    #for cell in range(ncpl):
    workingGrid.cell_arrays['cellHead']=layCellHeads
    workingVolume = workingGrid.extrude([0,0,1])
    for vert in range(nvert):
        workingVolume.points[vert][2] = modelVertZ[lay,vert]
        workingVolume.points[vert+nvert][2] = modelVertZ[lay+1,vert]
    workingVolume.point_arrays['vertexHead']=np.hstack([modelVertHead[lay],modelVertHead[lay+1]])
    workingVolume = workingVolume.threshold(value=-1.00000000e+30, scalars='cellHead')
    layerTubes[str(lay)] = workingVolume
layerBlocks = pv.MultiBlock(layerTubes)
totalModelGrid = layerBlocks.combine()
totalModelGrid.save('../vtk/totalModelHeads.vtk')

Working head cell for model grid: 100%|██████████| 6/6 [00:23<00:00,  3.92s/it]


In [ ]:
#head.shape
#centroidXYZArray.shape
#centroidHeads = pv.PolyData(centroidXYZArray)
#centroidHeads.point_arrays['head'] = headRshp.flatten()
#centroidHeads = centroidHeads(value=-1.00000000e+30, scalars='head')
#centroidHeads.save('../vtk/centroidHeads.vtk')
#centroidHeads.point_arrays?
#centroidXYZArray
layerTubes = {}
for lay in range(nlay):
    workingPCloud = pv.PolyData(centroidXYZDict[str(lay)])
    workingPCloud.point_arrays['head']=headRshp[lay]
    workingPCloud = workingPCloud.threshold(value=-1.00000000e+30, scalars='head')
    layerTubes[str(lay)]= workingPCloud
layerBlocks = pv.MultiBlock(layerTubes)
totalPCloud = layerBlocks.combine()
totalPCloud.save('../vtk/totalPCloud.vtk')

In [ ]:
workingGrid = baseGrid.copy()
workingGrid.points.shape

In [ ]:
totalModelGrid = layerBlocks.combine(merge_points=True)
totalModelGrid.points.shape[0]-99154*7

### for cell heads

### for water table

### for drain boundary

In [ ]:

drnPkg = mfmodel.get_package('drn_0')
drainTubes = {}
for index, drnCell in tqdm(enumerate(drnPkg.stress_period_data.array[0])):
    cellNumber = drnCell[0][1]
    cellLay = drnCell[0][0]
    cellGrid = mfmodel.disv.cell2d.array[cellNumber]
    vertexIndexList = [x for x in list(cellGrid)[4:] if x is not None]
    filterVertexIndexList = [len(vertexIndexList)] + list(range(len(vertexIndexList)))
    filterVertexIndexArray = np.array(filterVertexIndexList)
    filterVertexArray = np.array([list(verticesXYArray[vertex]) + [modelCellZ[cellLay,cellNumber]] for vertex in vertexIndexList])
    cellSurf = pv.PolyData(filterVertexArray, filterVertexIndexArray)
    cellSurf.cell_arrays['elevation'] = drnCell[1]
    cellSurf.cell_arrays['cond'] = drnCell[2]
    cellZ = modelCellZ[:,cellNumber]
    cellMesh = cellSurf.extrude([0, 0, cellZ[cellLay +1]-cellZ[cellLay]])
    drainTubes[str(index)] = cellMesh 
totalDrainsCells = pv.MultiBlock(drainTubes)
totalDrainsMerged = totalDrainsCells.combine()
totalDrainsMerged.save('../vtk/modelDrains.vtk')

In [ ]:
import pyvista as pv
import numpy as np
import os, flopy
import matplotlib.pyplot as plt
name = 'Model'
workspace = '../model'
mf_exe_name = '../exe/mf6'

sim = flopy.mf6.MFSimulation.load(sim_name=name, exe_name=mf_exe_name, sim_ws=workspace)

mfmodel = sim.get_model(model_name='model')

fname = os.path.join(workspace, name + '.hds')
hdobj = flopy.utils.HeadFile(fname, precision='double')
head = hdobj.get_data()
headRshp = head.reshape((head.shape[0],head.shape[2]))

mtop = np.ones([mfmodel.disv.nvert.array ])*mfmodel.disv.top[0]
verticesXYArray = np.dstack((mfmodel.disv.vertices.array.xv,mfmodel.disv.vertices.array.yv))[0]

In [ ]:
modelz = np.vstack((mfmodel.disv.top.array,mfmodel.disv.botm.array))
nlay = mfmodel.disv.nlay.array

In [ ]:
cellSurf = pv.PolyData()

In [ ]:
mtop